In [1]:
# Assess the similarity between generated DRRs and dataLoaded DRRs from PolyU
import torch
import numpy as np
import matplotlib.pyplot as plt
import os, sys, time, datetime
import pathlib
import pydicom.fileset
import pandas as pd

from scipy import ndimage, misc
import Metrics
import ImageProcessing

In [2]:
# Dataloader for DRR images
def load_DRR_image(csv_file_name = pathlib.Path("D:/data/CT-Covid-19-August2020/DRR_output/DRR_directory.csv"),
                   idx=0, image_type='source', flip=False):
    
    df = pd.read_csv(csv_file_name)
    image = np.load(df[image_type][idx])
    
    PixelSpacing_i = df["PixelSize_i"][idx]
    PixelSpacing_j = df["PixelSize_j"][idx]
    PixelSpacing = [PixelSpacing_i, PixelSpacing_j]
    
    if flip:
        image = np.flip(image, axis=0)
    return image, PixelSpacing



# Create a dataloader for CXR images
directoryPath = "D:\data\POLYU_COVID19_CXR_CT_Cohort1"
path_to_file = []
for root, dirs, files in os.walk(directoryPath):
    for name in files:
        if name=="DICOMDIR":
            path_to_file.append(os.path.join(root, name))

# Select a CXR directory
path_idx = 104
a_path = path_to_file[path_idx]

fs = pydicom.fileset.FileSet(a_path)

for instance in fs:
    if "1.2.840.10008.5.1.4.1.1.1" in instance.SOPClassUID:  # check if CR or DX image
        ds = instance.load()
        PixelSpacing_CXR = ds.ImagerPixelSpacing
        image = ds.pixel_array
        image_CXR = np.abs(np.amax(image)-image) # Dark is low intensity, bright is high intensity
        image_CXR = image_CXR/np.amax(image_CXR)
        
        # DRR
        image_DRR, PixelSpacing_DRR = load_DRR_image(idx = 2, flip=True)
        image_DRR = image_DRR/np.amax(image_DRR)
        # Upsample
        image_DRR = ndimage.zoom(image_DRR, (PixelSpacing_DRR[0]/PixelSpacing_CXR[0], PixelSpacing_DRR[1]/PixelSpacing_CXR[1]), order=0)
        
        

In [5]:
%matplotlib widget

fig, axs = plt.subplots(1, 2)
axs[0].imshow(image_DRR , cmap='bone')
axs[1].imshow(image_CXR , cmap='bone')

# Quantify similarity
image = image_DRR
means, variances = Metrics.patchMetrics(image)
#print(means)
SSIM = Metrics.SSIM(image_CXR, image_DRR, filter_kernel=(101,101), dynamic_range=1)
print(SSIM)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.7056848820250002
